In [53]:
import geopandas
from urllib.request import urlopen
import json
import pandas
from datetime import date, timedelta
import matplotlib
import matplotlib.pyplot as plt
import numpy as np




In [54]:
import fiona;
fiona.supported_drivers


{'ARCGEN': 'r',
 'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'rw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'GPSTrackMaker': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [55]:
def load_geodataset(remote_path, local_path, simplify=None, refresh_cache=False):
    df = None
    if not refresh_cache:
        try:
            df = geopandas.read_file(local_path)
        except Exception as e:
            print(e)
    
    if df is None:
        print(f"Loading {remote_path}")
        df = geopandas.read_file(remote_path)
        if simplify:
            df["geometry"] = df["geometry"].simplify(simplify)

        print(f"Saving to {local_path}")
        df.to_file(local_path)

    return df

In [56]:
def get_roads(refresh_cache=False):
    remote_path = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/RNF-FRR/files-fichiers/lrnf000r21a_e.zip"
    local_path = "data/roads.shp"
    return load_geodataset(remote_path, local_path, refresh_cache=refresh_cache)


In [57]:
def get_highways(refresh_cache=False, columns=["NAME", "TYPE", "CSDNAME_R", "PRNAME_L", "geometry"]):
    local_path = "data/highways.shp"
    
    highways = None
    if not refresh_cache:
        try:
            highways = geopandas.read_file(local_path)
        except Exception as e:
            print(e)
    
    if highways is None:
        roads = get_roads(refresh_cache)
        highways = roads.query('RANK <= "2"')
        print(f"Saving to {local_path}")
        highways.to_file(local_path)

    return highways[columns]


In [ ]:
# this is slow the first time you run it (downloads a 296 MB file), but it saves a smaller local version for future runs (123 MB)
highways = get_highways()
highways

Loading https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/RNF-FRR/files-fichiers/lrnf000r21a_e.zip


In [ ]:
highways.plot()

In [ ]:
def get_provinces(refresh_cache=False):
    remote_path = "https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/files-fichiers/2016/lpr_000b16a_e.zip"
    local_path = "data/provinces.shp"
    # simplify=1000 turns a 54 MB file to a 3 MB file
    return load_geodataset(remote_path, local_path, simplify=1000, refresh_cache=refresh_cache)

In [ ]:
provinces = get_provinces()
provinces.plot()

In [ ]:
# connector_type: [all, NEMA1450, NEMA515,NEMA520,J1772,J1772COMBO,CHADEMO,TESLA
def get_stations(connector_type="J1772COMBO", refresh_cache=False):
    remote_path=f"https://developer.nrel.gov/api/alt-fuel-stations/v1.geojson?api_key=JDgwj9XoVgfg8ryEgz1AOnnBbfoLATWjy4x6dOlv&country=CA&owner_type=all&cards_accepted=all&offset=0&fuel_type=ELEC&access=public&status=E&ev_charging_level=dc_fast&ev_connector_type={connector_type}&ev_network=all"
    local_path=f"data/stations_{connector_type}.shp"

    stations = None
    if not refresh_cache:
        try:
            stations = geopandas.read_file(local_path)
        except Exception as e:
            print(e)
    
    if stations is None:
        print(f"Loading {remote_path}")
        stations = geopandas.read_file(remote_path)
        stations.crs = "EPSG:4326"
        columns=["id", "open_date", "status_code", "station_name",
           "city", "state", "street_address", "ev_network", "geometry"]
        stations = stations.to_crs(highways.crs)[columns]
        print(f"Saving to {local_path}")
        print(stations)
        stations.to_file(local_path)

    return stations

    
    

In [ ]:
# Hyudani Kona - CCS Combo - 415 km
# Hyudani Ioniq 5 - CCS Combo - 354 km
# Hyudani Ioniq 5 Long Range - CCS Combo - 488 km
# Nissan Leaf - Chademo 349 km
# Telsa Model 3 Standard Range - 401 km
# Tesla Model 3 Extended Range - 518 km

connector_type= "J1772COMBO" # one of ["CHADEMO", "J1772COMBO", "TESLA"]
max_range = 415

In [ ]:
stations = get_stations(connector_type)
stations

In [ ]:
stations.to_crs("EPSG:4326").to_file(f"maps/stations_map_{connector_type}.geojson")

In [ ]:
stations.plot()

In [ ]:
ax = stations.plot(color='g', markersize=10)
highways.plot(ax=ax)

In [ ]:
# Method 1 for finding close stations - keep adding adjacent segments next to each EV
def find_stations_old_method():
    close_stations = geopandas.sjoin_nearest(highways, stations[["geometry"]], how='left', distance_col='charger_distance', max_distance=5000)
    close_stations = close_stations.drop(['index_right'], axis=1)

    found = close_stations[close_stations.charger_distance >= 0]
    not_found = close_stations[close_stations.charger_distance.isna()]
    f"{len(found)} / {len(not_found)}"

    changes = True
    tolerance = 0.1
    i = 0

    while changes and len(not_found) > 0:
        adjacent = geopandas.sjoin_nearest(not_found.drop(["charger_distance"], axis=1), found[["geometry", "charger_distance"]], how='left', distance_col='closest_distance', max_distance=tolerance)
        adjacent["charger_distance"] = adjacent["charger_distance"] + adjacent.geometry.length + adjacent["closest_distance"]
        adjacent = adjacent[found.columns]
        changes = len(adjacent[adjacent.charger_distance >= 0]) > 0
        found = pandas.concat([found, adjacent[adjacent.charger_distance >= 0]])
        not_found = adjacent[adjacent.charger_distance.isna()]
        i += 1
        if i % 100 == 0:
            print(f"{len(found)} / {len(not_found)}")
    print(f"{len(found)} / {len(not_found)}")

    not_found["charger_distance"] = 999999
    found = pandas.concat([found, not_found])
    return found

# found = find_stations_old_method()

In [ ]:
# Method 2 for finding close stations - look at adjacent stations and find minimum path to a charger
def calculate_ev_highways(stations, ev_highways=None):
    close_stations = geopandas.sjoin_nearest(highways, stations[["geometry"]], how='left', distance_col='charger_distance', max_distance=5000)
    close_stations = close_stations.drop(['index_right'], axis=1)

    if ev_highways is None:
        max_distance = 499999 # We don't care about roads more than 500 kms away from a station
        ev_highways = highways.copy()
        ev_highways["charger_distance"] = max_distance
    
    next_round = close_stations[close_stations["charger_distance"] >= 0]
    ev_highways.loc[next_round.index, "charger_distance"] = next_round["charger_distance"]

    changes = True
    tolerance = 0.1
    i = 0

    while changes:
        adjacent = geopandas.sjoin_nearest(ev_highways[["geometry", "charger_distance"]], next_round, how='left', distance_col="closest_distance", max_distance=tolerance)
        adjacent = adjacent[~adjacent["closest_distance"].isna()]
        adjacent["charger_distance"] = adjacent["charger_distance_right"] + adjacent.geometry.length + adjacent["closest_distance"]
        # Limit adjacents to shortest distance to a charger per segment
        adjacent.sort_values('charger_distance', inplace=True)
        adjacent = adjacent[~adjacent.index.duplicated()]
        # Only consider segments that are now shorter
        next_round = adjacent[adjacent["charger_distance"] < adjacent["charger_distance_left"] ]
        next_round = next_round[ev_highways.columns]
        ev_highways.loc[next_round.index, "charger_distance"] = next_round["charger_distance"]
        
        changes = len(next_round) > 0
        sum_distance = ev_highways["charger_distance"].sum()
        i += 1
        # if i % 100 == 0:
        #    print(f"changed: {len(next_round)} distances, sum distance={sum_distance}")

    return ev_highways



In [ ]:
def categorize_distance(dist):
    if dist * 2 <= min(50 * 1000, max_range * 1000 * 0.8 * 0.25):
        return "1 - Great"
    if dist * 2 <= min(100 * 1000, max_range * 1000 * 0.8 * 0.5):
        return "2 - Good"
    if dist * 2 <= min(200 * 1000, max_range * 1000 * 0.8 * 0.75):
        return "3 - Doable"
    if dist * 2 <= min(300 * 1000, max_range * 1000 * 0.8):
        return "4 - Hard"
    if dist * 2 <= min(350 * 1000, max_range * 1000 * 0.85):
        return "5 - Risky"
    return "6 - Inaccessible"

In [ ]:
def draw_map(filtered_stations,
             ev_highways,
             station_count_df,
             as_of_date,
             highway_options={"legend": False, "cmap": "YlGnBu_r"},
             station_options={"markersize":0.7, "color": "xkcd:dark blue"}
    ):
    ev_highways = ev_highways.copy()
    ev_highways['distance_cat'] = ev_highways['charger_distance'].apply(categorize_distance)
    ev_highways['length_km'] = ev_highways.length / 1000
    
    figsize = (15,11)
    ax = ev_highways.plot(column="distance_cat", figsize=figsize, **highway_options)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title("Canadian Highways with EV Fast Charging")

    cmap = matplotlib.cm.get_cmap('YlGnBu_r')
    axin1 = ax.inset_axes([0.75, 0.8, 0.25, 0.2])

    distance_histogram = ev_highways.groupby(["distance_cat"])["length_km"].sum().plot(ax=axin1, kind="barh", color=[cmap(0), cmap(1/5), cmap(2/5), cmap(3/5), cmap(4/5), cmap(5/5)], edgecolor="gray")
    distance_histogram.invert_yaxis()
    distance_histogram.set_xlim((0, 40000))
    distance_histogram.set_xticks([0, 10000, 20000, 30000 ])
    distance_histogram.set_xlabel("km")
    distance_histogram.set_ylabel("")
    category_lables = ["Great", "Good", "Doable", "Hard", "Risky", "Inaccessible"]
    distance_histogram.yaxis.set_major_formatter(lambda x, y: category_lables[x])
    

    if station_count_df is not None:
        axin2 = ax.inset_axes([0.045, 0.03, 0.3, 0.15])
        station_counts = station_count_df.copy()
        station_counts.loc[station_counts.index > as_of_date, "count"] = np.nan
        station_count_chart = station_counts.plot(ax=axin2, legend=False, title=f"{as_of_date} - {len(filtered_stations)} stations")
        station_count_chart.set_ylim((0, len(stations)))

    provinces.plot(ax=ax, figsize=figsize, color='xkcd:cool gray', zorder=0)
    provinces.boundary.plot(ax=ax, figsize=figsize, color='gray', linewidth=0.3, zorder=1)
    
    plot = filtered_stations.plot(ax=ax, figsize=figsize, **station_options)
    plot.get_figure().savefig(f"images/ev_highways_{connector_type}_{max_range}_{as_of_date}.png")
    
    plt.close('all')


In [ ]:
ev_highways = calculate_ev_highways(stations)
draw_map(stations, ev_highways, None, "today")
ev_highways['distance_cat'] = ev_highways['charger_distance'].apply(categorize_distance)
ev_highways['length_km'] = ev_highways.length / 1000
ev_highways.to_crs("EPSG:4326").to_file(f"maps/ev_highways_{connector_type}_{max_range}.geojson")


In [ ]:
def next_month(d):
    return date(d.year, d.month + 1, 1) if d.month < 12 else date(d.year + 1, 1, 1)

In [ ]:
stations["open_month"] = stations["open_date"].apply(lambda x: (date.fromisoformat(x) if x is not None else date.today()).replace(day=1))

start_date = stations["open_month"].min()
current_month = date(date.today().year, date.today().month, 1)
end_date = next_month(current_month)

station_count_df = pandas.DataFrame({"count": stations.groupby("open_month")["id"].count().reindex(pandas.date_range(start_date, end_date, freq='MS'), fill_value=0).cumsum()})
station_count_df

In [ ]:
as_of = date(2015, 1, 1)

ev_highways = None
while as_of <= end_date:
    as_of_date = as_of.strftime("%Y-%m-%d")
    print(as_of_date)
    
    filtered_stations = stations[stations["open_date"] <= as_of_date]
    
    ev_highways = calculate_ev_highways(filtered_stations, ev_highways)

    draw_map(filtered_stations, ev_highways, station_count_df, as_of_date)
    
    as_of = next_month(as_of)



In [ ]:
def province_good_and_great(ev_highways):
    ev_highways = ev_highways.copy()
    ev_highways['distance_cat'] = ev_highways['charger_distance'].apply(categorize_distance)
    ev_highways['length_km'] = ev_highways.length / 1000
    
    good_by_province = ev_highways.groupby(["PRNAME_L", "distance_cat"])["length_km"].sum()
    
    by_prov = pandas.DataFrame(good_by_province).reset_index(level=1).pivot_table(index=["PRNAME_L"], columns="distance_cat", values="length_km").fillna(0)
    by_prov["good_%"] = (by_prov["1 - Great"] + by_prov["2 - Good"]) / (by_prov["1 - Great"] + by_prov["2 - Good"] + by_prov["3 - Doable"] + by_prov["4 - Hard"] + by_prov["5 - Risky"] + by_prov["6 - Inaccessible"])
    return pandas.merge(provinces, by_prov, how="left", left_on=["PRNAME"], right_on=["PRNAME_L"]).fillna(0)


In [ ]:
by_prov = province_good_and_great(ev_highways)
by_prov


In [ ]:
ev_highways["good"] = ev_highways["charger_distance"] < 50*1000
ev_highways

In [ ]:
ax = by_prov.plot(column="good_%", cmap="Greens", edgecolor="black", linewidth=0.1, figsize=(15,11))
ax.set_title("Major Highways with Good EV Fast Charging")
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
def annotate_province_percentages(x):
    if x["PREABBR"] in ["Nvt.", "P.E.I.", "N.B.", "N.S."]:
        # Skip percentages for legibility
        return
    return ax.annotate(text=f"{x['good_%']:.0%}", xy=x.geometry.centroid.coords[0], ha="center")
by_prov.apply(annotate_province_percentages, axis=1);
ev_highways[ev_highways["good"] == True].plot(ax=ax, linewidth=0.2, color="xkcd:electric blue")
plot = ev_highways[ev_highways["good"] == False].plot(ax=ax, linewidth=0.2, color="xkcd:light grey")
plot.get_figure().savefig(f"images/canada_ev_highways_{connector_type}_good.png")
plot

In [ ]:
# new_stations
# Yorkton


new_stations_df = pandas.DataFrame({
    'City': ['Yorkton', 'Grande Prairie', 'Whitecourt', 'Boyle', 'Minnedosa', 'Prince Albert', 'McCleod Lake', 'Hanna', 'Kindersley', 'St. Barbe'],
    'Province': ['SK', 'AB', 'AB', 'AB', 'MB', 'SK', 'BC', 'AB', 'SK', 'NL'],
    'Latitude': [51.210516, 55.171337, 54.137583, 54.587165, 50.246068, 53.199422, 54.991843, 51.634314, 51.474346, 51.186568],
    'Longitude': [-102.448397, -118.821136, -115.686344, -112.803389, -99.839088, -105.759733, -123.034198, -111.942042, -109.168040, -56.779044],
    'Population': [16343, 63166, 9721, 845, 2449, 35926, 94, 2559, 4597, 135],
})

new_stations = geopandas.GeoDataFrame(new_stations_df, geometry=geopandas.points_from_xy(new_stations_df.Longitude, new_stations_df.Latitude))
new_stations.crs = "EPSG:4326"
new_stations = new_stations.to_crs(highways.crs)
new_stations

In [ ]:
fantasy_stations = pandas.concat([stations[["geometry"]], new_stations[["geometry"]]])
fantasy_ev_highways = calculate_ev_highways(fantasy_stations, ev_highways)


In [ ]:
draw_map(new_stations, fantasy_ev_highways, None, "fantasy", station_options={"markersize":15.0, "color": "xkcd:coral"})